In [1]:
import dash
from dash import Dash, dcc, html, State
from dash.dependencies import Input, Output
import dash_bootstrap_components as dbc
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
from dash.exceptions import PreventUpdate
import numpy as np


In [2]:
Telecom = pd.read_csv('./TelecomCustomerChurn.csv').iloc[:, 1:]
Telecom.head()

,Gender,SeniorCitizen,Partner,Dependents,Tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No,DSL,No,Yes,No,No,No,No,Monthly,Yes,Manual,29.85,29.85,No
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Manual,56.95,1889.5,No
2,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Monthly,Yes,Manual,53.85,108.15,Yes
3,Male,0,No,No,45,No,No,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Monthly,Yes,Manual,70.70,151.65,Yes


In [15]:


app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])


app.layout = dbc.Container([
    dbc.Row([
        dbc.Col(html.H1(
            "Telecom Customer Dashboard",
            className="text-center text-primary mb-4"
        ), width=12)
    ]),

    dcc.Tabs([

        
        dcc.Tab(label='General Overview', children=[
            html.Br(),
            html.H5("Paperless Billing and Gender"),
            dcc.Graph(
                id='global-overview-graph',
                figure=px.bar(
                    Telecom,
                    x='PaperlessBilling',
                    y='Gender',
                    title='Paperless Billing and Gender',
                    # markers=True
                )
            )
        ]),

       
        dcc.Tab(label='Gender Comparison', children=[
            html.Br(),
            html.Div([
                html.Label('Select Gender:', className='text-secondary'),
                dcc.Dropdown(
                    id='Gender-dropdown',
                    options=[
                        {'label': gender, 'value': gender}
                        for gender in Telecom['Gender'].unique()
                    ],
                    value=['Male', 'Female'],
                    multi=True
                )
            ], style={'width': '50%'}),
            html.Br(),
            dcc.Graph(id='Gender-comparison-graph')
        ]),
       
        dcc.Tab(label='Tenur by Internet service', children=[
            html.Br(),
            html.H5("Tenure by Internet Service"),
            dcc.Graph(
                id='new-cases-graph',
                figure=px.bar(
                    Telecom,
                    x='InternetService',
                    y='Tenure',
                    color='InternetService',
                    title='Tenure by Internet Service'
                )
            )
        ]),

       
        dcc.Tab(label='Total Charges by Contract Type', children=[
            html.Br(),
            html.H5(" Total Charges by Contract Type"),
            dcc.Graph(
                id='new-deaths-graph',
                figure=px.bar(
                    Telecom,
                    x='Contract',
                    y='TotalCharges',
                    color='Contract',
                    title='Total Charges by Contract'
                )
            )
        ]),

     
        dcc.Tab(label='Monthly Charges Analysis', children=[
            html.Br(),
            html.H5("Monthly Charges by Gender and Payment Method"),
            dcc.Graph(
                id='Charges-analysis-graph',
                figure=px.bar(
                    Telecom,
                    x='Gender',
                    y='MonthlyCharges',
                    color='PaymentMethod',
                    barmode='group',
                    title='Monthly Charges by Gender and Payment Method'
                )
            )
        ])
    ])
], fluid=True)


@app.callback(
    Output('Gender-comparison-graph', 'figure'),
    Input('Gender-dropdown', 'value')
)
def update_gender_comparison(selected_gender):
    filtered_df = Telecom[Telecom['Gender'].isin(selected_gender)]
    fig = px.bar(
        filtered_df,
        x='Contract',
        y='Gender',
        color='Gender',
        title='Contract Type Count by Gender',
    )
    return fig


if __name__ == '__main__':
    app.run(port=4444)
